# Biredctional LSTM

In [1]:
import json
import numpy as np
import re
import io
import nltk
import h5py
import keras as k
from keras.layers.embeddings import Embedding
from keras.layers import Input, Dense, Dropout, RepeatVector, Activation, merge, Lambda, Flatten, Reshape
from keras.layers import LSTM, Bidirectional, TimeDistributed, GRU
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras import optimizers
from keras.optimizers import Adam, RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import concatenate
from sklearn.metrics import f1_score

/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


CREATING EMBEDING LAYER WITH HELP OF WORD EMBEDDINGS: GLOVE

In [2]:
embeddings_index = {}
f = open( 'glove.6B.100d.txt',encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


IMPORT VECTORIZED FILES FOR CONTEXT, QUESTIONS, ANSWERS POST PREPROCESS OF DATA

In [3]:
context = h5py.File('context.h5','r')
questions = h5py.File('questions.h5','r')
answers = h5py.File('answers.h5','r')
ans_begin = h5py.File('begin.h5','r')
ans_end = h5py.File('end.h5','r')

In [4]:
c_data = context['context'][:]
qn_data = questions['questions'][:]
ans_data = answers['answers'][:]

In [5]:
begin_ans = ans_begin['begin'][:]
end_ans = ans_end['end'][:]

In [6]:
# loding vocabulary
word_index = np.load('words.npy').item()

In [7]:
embedding_matrix = np.zeros((len(word_index) + 1, 100))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [8]:
vocab_size = len(word_index) + 1
#embedding_vector_length = 50
batch_size = 64
max_span_begin = np.amax(begin_ans)
max_span_end = np.amax(end_ans)
train = 10000


In [9]:
print("Vocab Size")
vocab_size

Vocab Size


119616

MODEL CREATION

In [10]:
context_input = Input(shape=(700, ), dtype='int32', name='c_data')
embed_c = Embedding(input_dim=vocab_size, output_dim=100, weights=[embedding_matrix], 
              input_length=700, trainable=False)(context_input)
bidir_c = Bidirectional(LSTM(256, return_sequences=True, implementation=2), merge_mode='concat')(embed_c)
drop_1 = Dropout(0.5)(bidir_c)

In [11]:
ques_input = Input(shape=(100, ), dtype='int32', name='qn_data')
embed_q = Embedding(input_dim=vocab_size, output_dim=100, weights=[embedding_matrix], 
              input_length=100, trainable=False)(ques_input)
bidir_q = Bidirectional(LSTM(256, return_sequences=True, implementation=2), merge_mode='concat')(embed_q)
drop_2 = Dropout(0.5)(bidir_q)

In [12]:
merge_layer = concatenate([drop_1, drop_2], axis=1)
bidir_m = Bidirectional(LSTM(256, implementation=2), merge_mode='mul')(merge_layer)
drop_3 =  Dropout(0.5)(bidir_m)
softmax_1 = Dense(max_span_begin, activation='softmax')(bidir_m)
softmax_2 = Dense(max_span_end, activation='softmax')(bidir_m)
model = Model(inputs=[context_input, ques_input], outputs=[softmax_1, softmax_2])
opt = k.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
c_data (InputLayer)             (None, 700)          0                                            
__________________________________________________________________________________________________
qn_data (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 700, 100)     11961600    c_data[0][0]                     
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 100, 100)     11961600    qn_data[0][0]                    
__________________________________________________________________________________________________
bidirectio

In [13]:
model_history = model.fit([c_data[:train], qn_data[:train]],
                          [begin_ans[:train], end_ans[:train]], 
                          verbose=2,batch_size=batch_size, epochs=150)

Epoch 1/150
 - 546s - loss: 14.0659 - dense_1_loss: 7.0015 - dense_2_loss: 7.0644 - dense_1_acc: 0.0275 - dense_2_acc: 0.0046
Epoch 2/150
 - 527s - loss: 13.2918 - dense_1_loss: 6.6059 - dense_2_loss: 6.6859 - dense_1_acc: 0.0279 - dense_2_acc: 0.0060
Epoch 3/150
 - 527s - loss: 13.2521 - dense_1_loss: 6.5866 - dense_2_loss: 6.6655 - dense_1_acc: 0.0279 - dense_2_acc: 0.0051
Epoch 4/150
 - 527s - loss: 13.2388 - dense_1_loss: 6.5808 - dense_2_loss: 6.6580 - dense_1_acc: 0.0279 - dense_2_acc: 0.0069
Epoch 5/150
 - 527s - loss: 13.2213 - dense_1_loss: 6.5710 - dense_2_loss: 6.6503 - dense_1_acc: 0.0279 - dense_2_acc: 0.0066
Epoch 6/150
 - 526s - loss: 13.2075 - dense_1_loss: 6.5640 - dense_2_loss: 6.6435 - dense_1_acc: 0.0279 - dense_2_acc: 0.0071
Epoch 7/150
 - 528s - loss: 13.1986 - dense_1_loss: 6.5589 - dense_2_loss: 6.6397 - dense_1_acc: 0.0279 - dense_2_acc: 0.0070
Epoch 8/150
 - 527s - loss: 13.1902 - dense_1_loss: 6.5554 - dense_2_loss: 6.6348 - dense_1_acc: 0.0279 - dense_2_acc:

Epoch 66/150
 - 525s - loss: 13.1474 - dense_1_loss: 6.5344 - dense_2_loss: 6.6130 - dense_1_acc: 0.0279 - dense_2_acc: 0.0071
Epoch 67/150
 - 528s - loss: 13.1437 - dense_1_loss: 6.5317 - dense_2_loss: 6.6119 - dense_1_acc: 0.0279 - dense_2_acc: 0.0070
Epoch 68/150
 - 526s - loss: 13.1399 - dense_1_loss: 6.5296 - dense_2_loss: 6.6103 - dense_1_acc: 0.0279 - dense_2_acc: 0.0070
Epoch 69/150
 - 528s - loss: 13.1369 - dense_1_loss: 6.5284 - dense_2_loss: 6.6085 - dense_1_acc: 0.0279 - dense_2_acc: 0.0071
Epoch 70/150
 - 527s - loss: 13.1353 - dense_1_loss: 6.5279 - dense_2_loss: 6.6074 - dense_1_acc: 0.0279 - dense_2_acc: 0.0071
Epoch 71/150
 - 529s - loss: 13.1288 - dense_1_loss: 6.5243 - dense_2_loss: 6.6044 - dense_1_acc: 0.0279 - dense_2_acc: 0.0071
Epoch 72/150
 - 530s - loss: 13.1214 - dense_1_loss: 6.5200 - dense_2_loss: 6.6014 - dense_1_acc: 0.0279 - dense_2_acc: 0.0074
Epoch 73/150
 - 526s - loss: 13.1141 - dense_1_loss: 6.5169 - dense_2_loss: 6.5972 - dense_1_acc: 0.0279 - dens

KeyboardInterrupt: 

Model is stopped after 77 epochs as there seems to be no improvement in performance for more than 20 epochs. Further continuation would lead to loss in time and memory.